<a href="https://colab.research.google.com/github/mehmetbozdemir24/Tubitak_1505_Proje/blob/chunking/chunking_project1505.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pdfplumber python-docx python-pptx openpyxl pandas pypdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 16.2 MB/s eta 0:00:00


In [ ]:
#Kütüphanelerin yüklenmesi

import os
import pickle
import json
import pandas as pd
from typing import List, Dict, Any, Optional, Set
from pathlib import Path
from datetime import datetime
import re
import hashlib

try:
    import pdfplumber
except ImportError:
    os.system("pip install -q pdfplumber")
    import pdfplumber

try:
    from docx import Document as DocxDocument
except ImportError:
    os.system("pip install -q python-docx")
    from docx import Document as DocxDocument

try:
    from pptx import Presentation
except ImportError:
    os.system("pip install -q python-pptx")
    from pptx import Presentation

try:
    import openpyxl
except ImportError:
    os.system("pip install -q openpyxl")
    import openpyxl

In [ ]:
#metadata yapısı

class DocumentChunk:


    def __init__(self,
                 content: str,
                 dosya_adi: str,
                 sayfa_no: int,
                 chunk_no: int,
                 yetki_adi: List[str],
                 baslik: str = None,
                 satir_no: int = None,
                 sheet_adi: str = None,
                 slayt_no: int = None,
                 paragraf_no: int = None,
                 ek_bilgiler: Dict = None):

        self.content = content
        self.dosya_adi = dosya_adi
        self.sayfa_no = sayfa_no
        self.chunk_no = chunk_no
        self.yetki_adi = yetki_adi if isinstance(yetki_adi, list) else [yetki_adi]

        self.baslik = baslik
        self.satir_no = satir_no
        self.sheet_adi = sheet_adi
        self.slayt_no = slayt_no
        self.paragraf_no = paragraf_no
        self.ek_bilgiler = ek_bilgiler or {}

        self.chunk_id = self._generate_chunk_id()
        self.olusturma_tarihi = datetime.now().isoformat()
        self.dosya_turu = self._detect_file_type()

    def _generate_chunk_id(self):
        data = f"{self.dosya_adi}_{self.chunk_no}_{self.sayfa_no}"
        hash_val = hashlib.md5(data.encode()).hexdigest()[:8]
        return f"chunk_{self.chunk_no}_{hash_val}"

    def _detect_file_type(self):
        extension = Path(self.dosya_adi).suffix.lower()
        type_map = {
            '.pdf': 'pdf',
            '.docx': 'word',
            '.doc': 'word',
            '.xlsx': 'excel',
            '.xls': 'excel',
            '.pptx': 'powerpoint',
            '.ppt': 'powerpoint'
        }
        return type_map.get(extension, 'unknown')

    def get_metadata(self):
        """Zorunlu metadata formatında döndür"""
        metadata = {
            'dosya_adi': self.dosya_adi,
            'sayfa_no': self.sayfa_no,
            'chunk_no': self.chunk_no,
            'yetki_adi': self.yetki_adi,
            'dosya_turu': self.dosya_turu,
            'chunk_id': self.chunk_id,
            'olusturma_tarihi': self.olusturma_tarihi
        }

        if self.baslik:
            metadata['baslik'] = self.baslik
        if self.satir_no:
            metadata['satir_no'] = self.satir_no
        if self.sheet_adi:
            metadata['sheet_adi'] = self.sheet_adi
        if self.slayt_no:
            metadata['slayt_no'] = self.slayt_no
        if self.paragraf_no:
            metadata['paragraf_no'] = self.paragraf_no
        if self.ek_bilgiler:
            metadata['ek_bilgiler'] = self.ek_bilgiler

        return metadata

    def to_dict(self):
        return {
            'content': self.content,
            'metadata': self.get_metadata()
        }

    def has_permission(self, user: str) -> bool:
        return user in self.yetki_adi or 'admin' in self.yetki_adi

In [ ]:
#yetki yönetimi

class PermissionManager:

    def __init__(self):
        self.default_permissions = ['admin', 'user1']
        self.permission_rules = self._load_permission_rules()

    def _load_permission_rules(self):

        return {
            'gizli': ['admin'],
            'özel': ['admin', 'manager'],
            'mali': ['admin', 'finance'],
            'personel': ['admin', 'hr'],
            'genel': ['admin', 'user1', 'user2', 'user3'],
            'public': ['admin', 'user1', 'user2', 'user3'],
            'stratejik': ['admin', 'manager'],
            'yönetim': ['admin', 'manager'],
            'teknik': ['admin', 'tech', 'developer'],
        }

    def determine_permissions(self, content: str, file_name: str, baslik: str = None) -> List[str]:

        permissions = set(self.default_permissions)

        file_lower = file_name.lower()
        content_lower = content.lower()[:500] if content else ""
        baslik_lower = baslik.lower() if baslik else ""

        for keyword, perms in self.permission_rules.items():
            if keyword in file_lower or keyword in content_lower or keyword in baslik_lower:
                permissions.update(perms)

                if keyword in ['gizli', 'özel', 'mali', 'personel']:
                    permissions.discard('user1')
                    permissions.discard('user2')
                    permissions.discard('user3')

        return list(permissions)

In [ ]:
#dokümanlarda metin işleme kuralları

class TextProcessor:

    @staticmethod
    def fix_text_spacing(text: str) -> str:
        if not text:
            return text

        text = text.strip()

        text = re.sub(r'([a-zçğıöşü])([A-ZÇĞİÖŞÜ])', r'\1 \2', text)

        text = re.sub(r'([.!?])([A-ZÇĞİÖŞÜ])', r'\1 \2', text)

        text = re.sub(r'(\d\.)([A-ZÇĞİÖŞÜa-zçğıöşü])', r'\1 \2', text)

        text = re.sub(r',([A-ZÇĞİÖŞÜa-zçğıöşü])', r', \1', text)

        text = re.sub(r'\s+', ' ', text)

        return text.strip()

    @staticmethod
    def is_heading(text: str) -> bool:

        if not text or len(text.strip()) == 0:
            return False

        text = text.strip()

        patterns = [
            r'^(\d+\.)+\s*\w+',
            r'^(Bölüm|Kısım|Madde|Başlık)',
            r'^[A-Z][A-Z\s]+$',
        ]

        for pattern in patterns:
            if re.match(pattern, text):
                return True

        if len(text.split()) < 10:
            if text.isupper() or text.endswith(':'):
                return True

        return False

In [ ]:
#pdf dokümanlarının işlenmesi

class PDFProcessor:

    def __init__(self, permission_manager: PermissionManager):
        self.permission_manager = permission_manager
        self.text_processor = TextProcessor()
        self.chunk_counter = 0

    def process(self, file_path: str) -> List[DocumentChunk]:
        chunks = []
        self.chunk_counter = 0
        file_name = Path(file_path).name

        print(f"📄 PDF işleniyor: {file_name}")

        try:
            with pdfplumber.open(file_path) as pdf:
                print(f"   Toplam sayfa: {len(pdf.pages)}")

                current_heading = None
                current_content = []
                current_page = 1

                for page_num, page in enumerate(pdf.pages, 1):

                    text = page.extract_text(layout=True, use_text_flow=True) if hasattr(page, 'extract_text') else page.extract_text()

                    if not text:
                        continue

                    lines = text.split('\n')

                    for line in lines:
                        line = self.text_processor.fix_text_spacing(line.strip())

                        if not line:
                            continue


                        if self.text_processor.is_heading(line):

                            if current_heading and current_content:
                                self.chunk_counter += 1
                                content_text = ' '.join(current_content)
                                content_text = self.text_processor.fix_text_spacing(content_text)
                                full_content = f"{current_heading}\n\n{content_text}"

                                permissions = self.permission_manager.determine_permissions(
                                    full_content, file_name, current_heading
                                )

                                chunk = DocumentChunk(
                                    content=full_content,
                                    dosya_adi=file_name,
                                    sayfa_no=current_page,
                                    chunk_no=self.chunk_counter,
                                    yetki_adi=permissions,
                                    baslik=current_heading,
                                    ek_bilgiler={'metin_uzunlugu': len(full_content)}
                                )
                                chunks.append(chunk)

                            current_heading = line
                            current_content = []
                            current_page = page_num
                        else:
                            current_content.append(line)


                if current_heading and current_content:
                    self.chunk_counter += 1
                    content_text = ' '.join(current_content)
                    content_text = self.text_processor.fix_text_spacing(content_text)
                    full_content = f"{current_heading}\n\n{content_text}"

                    permissions = self.permission_manager.determine_permissions(
                        full_content, file_name, current_heading
                    )

                    chunk = DocumentChunk(
                        content=full_content,
                        dosya_adi=file_name,
                        sayfa_no=current_page,
                        chunk_no=self.chunk_counter,
                        yetki_adi=permissions,
                        baslik=current_heading,
                        ek_bilgiler={'metin_uzunlugu': len(full_content)}
                    )
                    chunks.append(chunk)

                if not chunks:
                    print("   Başlık bulunamadı, sayfa bazlı bölünüyor...")
                    for page_num, page in enumerate(pdf.pages, 1):
                        text = page.extract_text()
                        if text and text.strip():
                            self.chunk_counter += 1
                            text = self.text_processor.fix_text_spacing(text)

                            permissions = self.permission_manager.determine_permissions(
                                text, file_name, f"Sayfa {page_num}"
                            )

                            chunk = DocumentChunk(
                                content=text,
                                dosya_adi=file_name,
                                sayfa_no=page_num,
                                chunk_no=self.chunk_counter,
                                yetki_adi=permissions,
                                baslik=f"Sayfa {page_num}",
                                ek_bilgiler={'bolme_yontemi': 'sayfa_bazli'}
                            )
                            chunks.append(chunk)

        except Exception as e:
            print(f"   PDF hatası: {str(e)}")

        print(f"   Toplam chunk: {len(chunks)}")
        return chunks

In [ ]:
#excel dokümanlarının işlenmesi

class ExcelProcessor:

    def __init__(self, permission_manager: PermissionManager):
        self.permission_manager = permission_manager
        self.chunk_counter = 0

    def process(self, file_path: str) -> List[DocumentChunk]:
        chunks = []
        self.chunk_counter = 0
        file_name = Path(file_path).name

        print(f" Excel işleniyor: {file_name}")

        try:
            excel_file = pd.ExcelFile(file_path)

            for sheet_name in excel_file.sheet_names:
                df = excel_file.parse(sheet_name)
                print(f"   Sheet: {sheet_name} - {len(df)} satır")

                for idx, row in df.iterrows():

                    row_data = []
                    for col, val in row.items():
                        if pd.notna(val) and str(val).strip():
                            row_data.append(f"{col}: {str(val)}")

                    if row_data:
                        self.chunk_counter += 1
                        content = " | ".join(row_data)

                        permissions = self.permission_manager.determine_permissions(
                            content, file_name, sheet_name
                        )

                        chunk = DocumentChunk(
                            content=content,
                            dosya_adi=file_name,
                            sayfa_no=1,
                            chunk_no=self.chunk_counter,
                            yetki_adi=permissions,
                            baslik=sheet_name,
                            satir_no=idx + 1,
                            sheet_adi=sheet_name,
                            ek_bilgiler={'sutun_sayisi': len(row_data)}
                        )
                        chunks.append(chunk)

        except Exception as e:
            print(f"  Excel hatası: {str(e)}")

            try:
                wb = openpyxl.load_workbook(file_path, data_only=True)
                for sheet_idx, sheet_name in enumerate(wb.sheetnames, 1):
                    sheet = wb[sheet_name]
                    for row_idx, row in enumerate(sheet.iter_rows(values_only=True), 1):
                        content = " | ".join([str(c) for c in row if c])
                        if content.strip():
                            self.chunk_counter += 1

                            permissions = self.permission_manager.determine_permissions(
                                content, file_name, sheet_name
                            )

                            chunk = DocumentChunk(
                                content=content,
                                dosya_adi=file_name,
                                sayfa_no=sheet_idx,
                                chunk_no=self.chunk_counter,
                                yetki_adi=permissions,
                                baslik=sheet_name,
                                satir_no=row_idx,
                                sheet_adi=sheet_name
                            )
                            chunks.append(chunk)
            except Exception as e2:
                print(f"    Openpyxl hatası: {str(e2)}")

        print(f"   Toplam chunk: {len(chunks)}")
        return chunks

In [ ]:
#word dokümanlarının işlenmesi

class WordProcessor:

    def __init__(self, permission_manager: PermissionManager):
        self.permission_manager = permission_manager
        self.text_processor = TextProcessor()
        self.chunk_counter = 0

    def process(self, file_path: str) -> List[DocumentChunk]:

        chunks = []
        self.chunk_counter = 0
        file_name = Path(file_path).name

        print(f" Word işleniyor: {file_name}")

        try:
            doc = DocxDocument(file_path)
            current_heading = None
            current_content = []
            current_page = 1

            for para_idx, para in enumerate(doc.paragraphs):
                text = self.text_processor.fix_text_spacing(para.text.strip())
                if not text:
                    continue

                estimated_page = (para_idx // 40) + 1

                is_heading_style = para.style.name.startswith('Heading') if para.style else False
                is_heading_text = self.text_processor.is_heading(text)

                if is_heading_style or is_heading_text:

                    if current_heading and current_content:
                        self.chunk_counter += 1
                        content_text = ' '.join(current_content)
                        content_text = self.text_processor.fix_text_spacing(content_text)
                        full_content = f"{current_heading}\n\n{content_text}"

                        permissions = self.permission_manager.determine_permissions(
                            full_content, file_name, current_heading
                        )

                        chunk = DocumentChunk(
                            content=full_content,
                            dosya_adi=file_name,
                            sayfa_no=current_page,
                            chunk_no=self.chunk_counter,
                            yetki_adi=permissions,
                            baslik=current_heading,
                            paragraf_no=para_idx,
                            ek_bilgiler={'stil': para.style.name if para.style else 'Normal'}
                        )
                        chunks.append(chunk)

                    current_heading = text
                    current_content = []
                    current_page = estimated_page
                else:
                    current_content.append(text)

            if current_heading and current_content:
                self.chunk_counter += 1
                content_text = ' '.join(current_content)
                content_text = self.text_processor.fix_text_spacing(content_text)
                full_content = f"{current_heading}\n\n{content_text}"

                permissions = self.permission_manager.determine_permissions(
                    full_content, file_name, current_heading
                )

                chunk = DocumentChunk(
                    content=full_content,
                    dosya_adi=file_name,
                    sayfa_no=current_page,
                    chunk_no=self.chunk_counter,
                    yetki_adi=permissions,
                    baslik=current_heading
                )
                chunks.append(chunk)

            if not chunks:
                print("    Başlık bulunamadı, paragraf bazlı bölünüyor...")
                for idx, para in enumerate(doc.paragraphs):
                    if para.text.strip():
                        self.chunk_counter += 1
                        text = self.text_processor.fix_text_spacing(para.text)
                        estimated_page = (idx // 40) + 1

                        permissions = self.permission_manager.determine_permissions(
                            text, file_name, f"Paragraf {idx+1}"
                        )

                        chunk = DocumentChunk(
                            content=text,
                            dosya_adi=file_name,
                            sayfa_no=estimated_page,
                            chunk_no=self.chunk_counter,
                            yetki_adi=permissions,
                            baslik=f"Paragraf {idx+1}",
                            paragraf_no=idx + 1
                        )
                        chunks.append(chunk)

        except Exception as e:
            print(f"   Word hatası: {str(e)}")

        print(f"   Toplam chunk: {len(chunks)}")
        return chunks

In [ ]:
#powerpoint dokümanlarının işlenmesi

class PowerPointProcessor:

    def __init__(self, permission_manager: PermissionManager):
        self.permission_manager = permission_manager
        self.chunk_counter = 0

    def process(self, file_path: str) -> List[DocumentChunk]:

        chunks = []
        self.chunk_counter = 0
        file_name = Path(file_path).name

        print(f" PowerPoint işleniyor: {file_name}")

        try:
            prs = Presentation(file_path)
            print(f"   Toplam slayt: {len(prs.slides)}")

            for slide_idx, slide in enumerate(prs.slides, 1):
                self.chunk_counter += 1
                slide_title = None
                slide_content = []

                for shape in slide.shapes:
                    if hasattr(shape, 'text'):
                        text = shape.text.strip()
                        if not text:
                            continue

                        if slide_title is None and len(text.split()) < 10:
                            slide_title = text
                        else:
                            slide_content.append(text)

                if not slide_title:
                    slide_title = f"Slayt {slide_idx}"

                content = f"{slide_title}\n\n"
                if slide_content:
                    content += "\n".join(slide_content)

                permissions = self.permission_manager.determine_permissions(
                    content, file_name, slide_title
                )

                chunk = DocumentChunk(
                    content=content,
                    dosya_adi=file_name,
                    sayfa_no=slide_idx,
                    chunk_no=self.chunk_counter,
                    yetki_adi=permissions,
                    baslik=slide_title,
                    slayt_no=slide_idx,
                    ek_bilgiler={'icerik_sayisi': len(slide_content)}
                )
                chunks.append(chunk)

        except Exception as e:
            print(f"  PowerPoint hatası: {str(e)}")

        print(f"  Toplam chunk: {len(chunks)}")
        return chunks

In [ ]:
#ana doküman işleyici

class DocumentChunker:


    def __init__(self, output_dir: str = "./chunks"):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(exist_ok=True)

        # İşleyicileri başlat
        self.permission_manager = PermissionManager()
        self.pdf_processor = PDFProcessor(self.permission_manager)
        self.excel_processor = ExcelProcessor(self.permission_manager)
        self.word_processor = WordProcessor(self.permission_manager)
        self.ppt_processor = PowerPointProcessor(self.permission_manager)

        print(f" Çıktı dizini: {self.output_dir}")

    def process_file(self, file_path: str, custom_permissions: List[str] = None) -> List[DocumentChunk]:
        file_path = Path(file_path)

        if not file_path.exists():
            print(f" Dosya bulunamadı: {file_path}")
            return []

        extension = file_path.suffix.lower()

        print(f"\n{'='*60}")
        print(f" İşleniyor: {file_path.name}")
        print(f" Tür: {extension}")
        print(f"{'='*60}")

        chunks = []

        if extension in ['.xlsx', '.xls']:
            chunks = self.excel_processor.process(str(file_path))
        elif extension == '.pdf':
            chunks = self.pdf_processor.process(str(file_path))
        elif extension in ['.docx', '.doc']:
            chunks = self.word_processor.process(str(file_path))
        elif extension in ['.pptx', '.ppt']:
            chunks = self.ppt_processor.process(str(file_path))
        else:
            print(f" Desteklenmeyen dosya türü: {extension}")
            return []

        if custom_permissions and chunks:
            for chunk in chunks:
                chunk.yetki_adi = custom_permissions

        return chunks

    def save_chunks(self, chunks: List[DocumentChunk], output_name: str):

        if not chunks:
            print(" Kaydedilecek chunk yok!")
            return None

        chunks_dict = [chunk.to_dict() for chunk in chunks]

        pkl_path = self.output_dir / f"{output_name}.pkl"
        with open(pkl_path, 'wb') as f:
            pickle.dump(chunks_dict, f)

        meta_path = self.output_dir / f"{output_name}_metadata.json"
        metadata_only = []
        for chunk in chunks:
            meta = chunk.get_metadata()
            meta.pop('content', None)
            metadata_only.append(meta)

        with open(meta_path, 'w', encoding='utf-8') as f:
            json.dump(metadata_only, f, ensure_ascii=False, indent=2)

        print(f"\n Kaydedildi:")
        print(f"    Data: {pkl_path}")
        print(f"    Metadata: {meta_path}")
        print(f"    Chunk sayısı: {len(chunks)}")

        permission_summary = {}
        for chunk in chunks:
            for perm in chunk.yetki_adi:
                permission_summary[perm] = permission_summary.get(perm, 0) + 1

        print(f"    Yetki dağılımı: {permission_summary}")

        return pkl_path

    def load_chunks(self, pkl_file: str, user: str = None) -> List[Dict]:

        with open(pkl_file, 'rb') as f:
            chunks_dict = pickle.load(f)

        if user:

            filtered_chunks = []
            denied_count = 0

            for chunk_data in chunks_dict:
                metadata = chunk_data.get('metadata', {})
                permissions = metadata.get('yetki_adi', [])

                if user in permissions or 'admin' in permissions:
                    filtered_chunks.append(chunk_data)
                else:
                    denied_count += 1

            print(f"\n Yetki Raporu - Kullanıcı: {user}")
            print(f"    Erişilebilir: {len(filtered_chunks)} chunk")
            print(f"    Erişim engellendi: {denied_count} chunk")

            return filtered_chunks

        return chunks_dict

    def display_chunks(self, chunks: List, max_display: int = 3, user: str = None):

        if not chunks:
            print(" Görüntülenecek chunk yok!")
            return

        display_chunks = []
        for chunk in chunks:
            if isinstance(chunk, DocumentChunk):
                if not user or chunk.has_permission(user):
                    display_chunks.append(chunk)
            else:
                metadata = chunk.get('metadata', {})
                permissions = metadata.get('yetki_adi', [])
                if not user or user in permissions or 'admin' in permissions:
                    display_chunks.append(chunk)

        print(f"\n Chunk Görüntüleme ({len(display_chunks)} chunk)")
        print("─" * 60)

        for i, chunk in enumerate(display_chunks[:max_display]):
            if isinstance(chunk, DocumentChunk):
                metadata = chunk.get_metadata()
                content = chunk.content
            else:
                metadata = chunk.get('metadata', {})
                content = chunk.get('content', '')

            print(f"\n[Chunk #{metadata.get('chunk_no', i+1)}]")
            print(f" Dosya: {metadata.get('dosya_adi', 'N/A')}")
            print(f" Sayfa: {metadata.get('sayfa_no', 'N/A')}")
            print(f" Yetkiler: {', '.join(metadata.get('yetki_adi', []))}")

            if metadata.get('baslik'):
                print(f" Başlık: {metadata.get('baslik')}")
            if metadata.get('sheet_adi'):
                print(f" Sheet: {metadata.get('sheet_adi')}")
            if metadata.get('satir_no'):
                print(f" Satır: {metadata.get('satir_no')}")

            print(f" İçerik (ilk 200 karakter):")
            print(f"   {content[:200]}...")
            print("─" * 40)

    def process_directory(self, directory: str, custom_permissions: List[str] = None):

        directory = Path(directory)

        if not directory.exists():
            print(f" Dizin bulunamadı: {directory}")
            return {}

        supported = ['.xlsx', '.xls', '.pdf', '.docx', '.doc', '.pptx', '.ppt']
        all_results = {}

        print(f"\n Dizin işleniyor: {directory}")
        files = list(directory.glob("*"))
        supported_files = [f for f in files if f.suffix.lower() in supported]

        print(f" Bulunan dosyalar: {len(supported_files)}")

        for file_path in supported_files:
            chunks = self.process_file(str(file_path), custom_permissions)
            if chunks:
                output_name = file_path.stem
                self.save_chunks(chunks, output_name)
                all_results[file_path.name] = chunks


        print(f"\n ÖZET:")
        print("=" * 60)
        total_chunks = 0
        for filename, chunks in all_results.items():
            count = len(chunks)
            total_chunks += count
            print(f"   {filename}: {count} chunk")
        print(f"   Toplam: {total_chunks} chunk")

        return all_results

In [ ]:
#chunk görüntüleme

class ChunkViewer:

    def __init__(self, chunk_dir: str = "./chunks"):
        self.chunk_dir = Path(chunk_dir)

    def view_metadata_summary(self, pkl_file: str):

        with open(pkl_file, 'rb') as f:
            chunks = pickle.load(f)

        print(f"\n{'='*80}")
        print(f" Meta-data Özeti: {Path(pkl_file).name}")
        print(f"{'='*80}")

        total_chunks = len(chunks)
        file_types = {}
        permissions_stats = {}
        page_range = {'min': float('inf'), 'max': 0}

        for chunk in chunks:
            metadata = chunk.get('metadata', {})

            file_type = metadata.get('dosya_turu', 'unknown')
            file_types[file_type] = file_types.get(file_type, 0) + 1

            for perm in metadata.get('yetki_adi', []):
                permissions_stats[perm] = permissions_stats.get(perm, 0) + 1

            page_no = metadata.get('sayfa_no', 0)
            if page_no:
                page_range['min'] = min(page_range['min'], page_no)
                page_range['max'] = max(page_range['max'], page_no)

        print(f"\n İstatistikler:")
        print(f"   • Toplam chunk: {total_chunks}")
        print(f"   • Dosya türleri: {file_types}")
        print(f"   • Sayfa aralığı: {page_range['min']}-{page_range['max']}")

        print(f"\n Yetki Dağılımı:")
        for perm, count in sorted(permissions_stats.items(), key=lambda x: x[1], reverse=True):
            percentage = (count / total_chunks) * 100
            print(f"   • {perm}: {count} chunk ({percentage:.1f}%)")

    def export_report(self, pkl_file: str, output_file: str = None):

        with open(pkl_file, 'rb') as f:
            chunks = pickle.load(f)

        if not output_file:
            output_file = Path(pkl_file).stem + "_report.txt"

        output_path = self.chunk_dir / output_file

        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write(f"Doküman Chunk Raporu\n")
            f.write(f"Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Dosya: {Path(pkl_file).name}\n")
            f.write("="*80 + "\n\n")

            for chunk in chunks:
                metadata = chunk.get('metadata', {})
                content = chunk.get('content', '')

                f.write(f"\nChunk #{metadata.get('chunk_no', 'N/A')}\n")
                f.write("-"*40 + "\n")
                f.write(f"Dosya: {metadata.get('dosya_adi', 'N/A')}\n")
                f.write(f"Sayfa: {metadata.get('sayfa_no', 'N/A')}\n")
                f.write(f"Yetkiler: {', '.join(metadata.get('yetki_adi', []))}\n")

                if metadata.get('baslik'):
                    f.write(f"Başlık: {metadata.get('baslik')}\n")

                f.write(f"\nİçerik:\n{content[:500]}...\n")
                f.write("="*60 + "\n")

        print(f" Rapor oluşturuldu: {output_path}")
        return output_path


In [ ]:
#demo kullanım

def demo():

    # Chunker başlat
    chunker = DocumentChunker(output_dir="/content/drive/MyDrive/1505 Chunking/document_chunks")

    # Test dosyası işle
    test_file = "/content/drive/MyDrive/1505 Chunking/Doküman/pdf/Bilimp_Guncelleme_Dokumani_Ornek_Sablon.pdf"

    if Path(test_file).exists():

        chunks = chunker.process_file(test_file)
        # Veya özel yetkilerle
        # chunks = chunker.process_file(test_file, custom_permissions=['admin', 'manager'])

        if chunks:

            chunker.save_chunks(chunks, "test_output")

            chunker.display_chunks(chunks, max_display=3)

            # Belirli kullanıcı için yükle
            user_chunks = chunker.load_chunks("./document_chunks/test_output.pkl", user="user1")
            print(f"\nuser1 için {len(user_chunks)} chunk erişilebilir")

    # Viewer kullanımı
    viewer = ChunkViewer("./document_chunks")

    # Metadata özeti
    if Path("/content/drive/MyDrive/1505 Chunking/document_chunks/test_output.pkl").exists():
        viewer.view_metadata_summary("/content/drive/MyDrive/1505 Chunking/document_chunks/test_output.pkl")

        # Rapor oluştur
        viewer.export_report("/content/drive/MyDrive/1505 Chunking/document_chunks/test_output.pkl")

    print("\n İşlem tamamlandı!")

# Google Colab için
def mount_drive():
    """Google Drive bağla"""
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print(" Google Drive bağlandı!")
        return True
    except:
        print(" Google Drive bağlanamadı!")
        return False

if __name__ == "__main__":
    demo()


=================== DOKÜMAN CHUNKING SİSTEMİ (YENİ METADATA) ===================
📁 Çıktı dizini: /content/drive/MyDrive/1505 Chunking/document_chunks

📄 İşleniyor: Bilimp_Guncelleme_Dokumani_Ornek_Sablon.pdf
📂 Tür: .pdf
📄 PDF işleniyor: Bilimp_Guncelleme_Dokumani_Ornek_Sablon.pdf
   Toplam sayfa: 3
   ✅ Toplam chunk: 6

✅ Kaydedildi:
   📦 Data: /content/drive/MyDrive/1505 Chunking/document_chunks/test_output.pkl
   📋 Metadata: /content/drive/MyDrive/1505 Chunking/document_chunks/test_output_metadata.json
   📊 Chunk sayısı: 6
   🔐 Yetki dağılımı: {'admin': 6, 'user1': 6}

📄 Chunk Görüntüleme (6 chunk)
────────────────────────────────────────────────────────────

[Chunk #1]
📁 Dosya: Bilimp_Guncelleme_Dokumani_Ornek_Sablon.pdf
📄 Sayfa: 1
🔐 Yetkiler: admin, user1
📌 Başlık: 3. Proje Aracında Proje Başlatma ve Yatırım Talep form sayfalarında zorunlu alanlarla ilgili geliştirme
📝 İçerik (ilk 200 karakter):
   3. Proje Aracında Proje Başlatma ve Yatırım Talep form sayfalarında zorunlu alanlar

In [ ]:
#Toplu Chunk İşlemi

class BatchChunker:
    """Klasördeki tüm dosyaları toplu olarak chunk'lara ayır"""

    def __init__(self, output_dir: str = "./chunks"):
        self.chunker = DocumentChunker(output_dir=output_dir)
        self.output_dir = Path(output_dir)
        self.results = {}

    def process_folder(self,
                       folder_path: str,
                       custom_permissions: List[str] = None,
                       recursive: bool = False,
                       save_individual: bool = True,
                       save_combined: bool = False):

        folder_path = Path(folder_path)

        if not folder_path.exists():
            print(f" Klasör bulunamadı: {folder_path}")
            return None

        print("\n" + "="*80)
        print(f" Toplu Chunking İşlemi ".center(80, "="))
        print(f" Klasör: {folder_path}")
        print(f" Recursive: {'Evet' if recursive else 'Hayır'}")
        print(f" Kayıt modu: {'Bireysel' if save_individual else ''}{' + Birleşik' if save_combined else ''}")

        supported_extensions = ['.pdf', '.docx', '.doc', '.xlsx', '.xls', '.pptx', '.ppt']

        files_to_process = []
        if recursive:
            for ext in supported_extensions:
                files_to_process.extend(list(folder_path.rglob(f"*{ext}")))
        else:
            for ext in supported_extensions:
                files_to_process.extend(list(folder_path.glob(f"*{ext}")))

        if not files_to_process:
            print(" İşlenecek dosya bulunamadı!")
            return None

        print(f"\n Bulunan dosyalar: {len(files_to_process)}")

        file_types = {}
        for file in files_to_process:
            ext = file.suffix.lower()
            file_types[ext] = file_types.get(ext, 0) + 1

        print(f" Dosya türü dağılımı:")
        for ext, count in sorted(file_types.items()):
            print(f"   • {ext}: {count} dosya")

        all_chunks = []
        processed_count = 0
        failed_count = 0


        for idx, file_path in enumerate(files_to_process, 1):
            try:
                print(f"\n[{idx}/{len(files_to_process)}] İşleniyor: {file_path.name}")

                # Dosyayı işle
                chunks = self.chunker.process_file(str(file_path), custom_permissions)

                if chunks:
                    processed_count += 1

                    self.results[file_path.name] = {
                        'path': str(file_path),
                        'chunks': chunks,
                        'chunk_count': len(chunks),
                        'status': 'success'
                    }

                    if save_individual:
                        output_name = file_path.stem
                        counter = 1
                        original_name = output_name
                        while (self.output_dir / f"{output_name}.pkl").exists():
                            output_name = f"{original_name}_{counter}"
                            counter += 1

                        self.chunker.save_chunks(chunks, output_name)

                    if save_combined:
                        all_chunks.extend(chunks)

                    print(f"    Başarılı: {len(chunks)} chunk oluşturuldu")
                else:
                    failed_count += 1
                    self.results[file_path.name] = {
                        'path': str(file_path),
                        'chunks': [],
                        'chunk_count': 0,
                        'status': 'no_chunks'
                    }
                    print(f"    Chunk oluşturulamadı")

            except Exception as e:
                failed_count += 1
                self.results[file_path.name] = {
                    'path': str(file_path),
                    'chunks': [],
                    'chunk_count': 0,
                    'status': 'error',
                    'error': str(e)
                }
                print(f"    Hata: {str(e)}")

        if save_combined and all_chunks:
            print(f"\n Birleşik chunk dosyası oluşturuluyor...")
            combined_name = f"combined_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            self.chunker.save_chunks(all_chunks, combined_name)

        # Özet
        self._print_summary(processed_count, failed_count, all_chunks if save_combined else None)

        return self.results

    def _print_summary(self, processed_count, failed_count, all_chunks=None):

        print(f"\n Dosya İstatistikleri:")
        print(f"    Başarılı: {processed_count}")
        print(f"    Başarısız: {failed_count}")
        print(f"    Toplam: {processed_count + failed_count}")

        # Chunk istatistikleri
        total_chunks = sum(r['chunk_count'] for r in self.results.values())
        print(f"\n Chunk İstatistikleri:")
        print(f"   Toplam chunk: {total_chunks}")

        if total_chunks > 0:
            avg_chunks = total_chunks / processed_count if processed_count > 0 else 0
            print(f"   • Dosya başına ortalama: {avg_chunks:.1f} chunk")

        # Dosya türü bazında
        type_stats = {}
        for result in self.results.values():
            file_path = Path(result['path'])
            ext = file_path.suffix.lower()
            if ext not in type_stats:
                type_stats[ext] = {'files': 0, 'chunks': 0}
            type_stats[ext]['files'] += 1
            type_stats[ext]['chunks'] += result['chunk_count']

        print(f"\n Dosya Türü Bazında:")
        for ext, stats in sorted(type_stats.items()):
            print(f"   • {ext}: {stats['files']} dosya, {stats['chunks']} chunk")

        if all_chunks:
            permission_stats = {}
            for chunk in all_chunks:
                for perm in chunk.yetki_adi:
                    permission_stats[perm] = permission_stats.get(perm, 0) + 1

            print(f"\n Yetki Dağılımı:")
            for perm, count in sorted(permission_stats.items(), key=lambda x: x[1], reverse=True):
                percentage = (count / len(all_chunks)) * 100
                print(f"   • {perm}: {count} chunk ({percentage:.1f}%)")


    def export_batch_report(self, output_file: str = None):

        if not self.results:
            print(" Rapor için veri yok!")
            return None

        if not output_file:
            output_file = f"batch_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"

        output_path = self.output_dir / output_file

        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write("Toplu Doküman Chunking Raporu\n")
            f.write(f"Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("="*80 + "\n\n")

            f.write(" Başarılı Dosyalar:\n")
            f.write("-"*80 + "\n")
            for filename, result in self.results.items():
                if result['status'] == 'success':
                    f.write(f"\n {filename}\n")
                    f.write(f"   Chunk sayısı: {result['chunk_count']}\n")
                    f.write(f"   Dosya yolu: {result['path']}\n")

            f.write("\n\n Başarısız Dosyalar:\n")
            f.write("-"*80 + "\n")
            for filename, result in self.results.items():
                if result['status'] != 'success':
                    f.write(f"\n {filename}\n")
                    f.write(f"   Durum: {result['status']}\n")
                    if 'error' in result:
                        f.write(f"   Hata: {result['error']}\n")

            # İstatistikler
            total_files = len(self.results)
            successful = sum(1 for r in self.results.values() if r['status'] == 'success')
            total_chunks = sum(r['chunk_count'] for r in self.results.values())

            f.write("\n\n İstatistikler:\n")
            f.write("-"*80 + "\n")
            f.write(f"Toplam dosya: {total_files}\n")
            f.write(f"Başarılı: {successful}\n")
            f.write(f"Başarısız: {total_files - successful}\n")
            f.write(f"Toplam chunk: {total_chunks}\n")
            if successful > 0:
                f.write(f"Dosya başına ortalama: {total_chunks/successful:.1f} chunk\n")

        print(f" Rapor oluşturuldu: {output_path}")
        return output_path

    def get_failed_files(self):
        failed = []
        for filename, result in self.results.items():
            if result['status'] != 'success':
                failed.append({
                    'filename': filename,
                    'path': result['path'],
                    'status': result['status'],
                    'error': result.get('error', 'N/A')
                })
        return failed

    def retry_failed(self, custom_permissions: List[str] = None):
        """Başarısız dosyaları tekrar işle"""
        failed = self.get_failed_files()

        if not failed:
            print(" Başarısız dosya yok!")
            return

        print(f"\n {len(failed)} başarısız dosya tekrar işleniyor...")

        for item in failed:
            try:
                print(f"\n Yeniden deneniyor: {item['filename']}")
                chunks = self.chunker.process_file(item['path'], custom_permissions)

                if chunks:
                    self.results[item['filename']] = {
                        'path': item['path'],
                        'chunks': chunks,
                        'chunk_count': len(chunks),
                        'status': 'success'
                    }
                    print(f"    Başarılı: {len(chunks)} chunk")
                else:
                    print(f"    Hala chunk oluşturulamadı")

            except Exception as e:
                print(f"    Hata devam ediyor: {str(e)}")


In [ ]:
def demo_batch_processing():

    # Batch chunker
    batch_chunker = BatchChunker(output_dir="./batch_chunks")

    folder_path = "/content/drive/MyDrive/1505 Chunking"

    if Path(folder_path).exists():
        results = batch_chunker.process_folder(
            folder_path=folder_path,
            recursive=False,
            save_individual=True,
            save_combined=True
        )

        batch_chunker.export_batch_report()

        failed = batch_chunker.get_failed_files()
        if failed:
            print(f"\n Başarısız dosyalar:")
            for f in failed:
                print(f"   • {f['filename']}: {f['error']}")

            batch_chunker.retry_failed()

    results = batch_chunker.process_folder(
        folder_path="/content/drive/MyDrive/1505 Chunking",
        recursive=True,
        custom_permissions=['admin', 'manager'],
        save_individual=True,
        save_combined=False
    )

    print("\n Toplu işlem tamamlandı!")


def quick_batch_chunk(folder_path: str,
                      output_dir: str = "/content/drive/MyDrive/1505 Chunking/document_chunks",
                      recursive: bool = False):
    batch = BatchChunker(output_dir=output_dir)
    results = batch.process_folder(
        folder_path=folder_path,
        recursive=recursive,
        save_individual=True,
        save_combined=True
    )
    batch.export_batch_report()
    return batch


batch = quick_batch_chunk("/content/drive/MyDrive/1505 Chunking/Doküman/powerpoint", recursive=True)

📁 Çıktı dizini: /content/drive/MyDrive/1505 Chunking/document_chunks

============================ TOPLU CHUNKING İŞLEMİ =============================
📁 Klasör: /content/drive/MyDrive/1505 Chunking/Doküman/powerpoint
🔄 Recursive: Evet
💾 Kayıt modu: Bireysel + Birleşik

📊 Bulunan dosyalar: 1
📈 Dosya türü dağılımı:
   • .pptx: 1 dosya

⏳ İşleme başlıyor...

[1/1] İşleniyor: Bilimp_Satis_Sunum_Ornek_Sablon.pptx

📄 İşleniyor: Bilimp_Satis_Sunum_Ornek_Sablon.pptx
📂 Tür: .pptx
📊 PowerPoint işleniyor: Bilimp_Satis_Sunum_Ornek_Sablon.pptx
   Toplam slayt: 7
   ✅ Toplam chunk: 7

✅ Kaydedildi:
   📦 Data: /content/drive/MyDrive/1505 Chunking/document_chunks/Bilimp_Satis_Sunum_Ornek_Sablon.pkl
   📋 Metadata: /content/drive/MyDrive/1505 Chunking/document_chunks/Bilimp_Satis_Sunum_Ornek_Sablon_metadata.json
   📊 Chunk sayısı: 7
   🔐 Yetki dağılımı: {'admin': 7, 'user1': 6, 'manager': 1, 'hr': 1}
   ✅ Başarılı: 7 chunk oluşturuldu

📦 Birleşik chunk dosyası oluşturuluyor...

✅ Kaydedildi:
   📦 Data:

In [ ]:
#pkl dosya gösterimi

import pickle

def show_pkl(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)

    print(f"Toplam chunk: {len(data)}\n")

    for i, chunk in enumerate(data[:10]):
        print(f"[Chunk {i+1}]")
        print(f"Dosya: {chunk['metadata']['dosya_adi']}")
        print(f"Sayfa: {chunk['metadata']['sayfa_no']}")
        print(f"Yetki: {chunk['metadata']['yetki_adi']}")
        print(f"İçerik: {chunk['content'][:200]}...\n")


show_pkl("/content/drive/MyDrive/1505 Chunking/document_chunks/Aylik_Satis_Ozet_Ornek_Sablon.pkl")

Toplam chunk: 96

[Chunk 1]
Dosya: Aylik_Satis_Ozet_Ornek_Sablon.xlsx
Sayfa: 1
Yetki: ['admin', 'user1']
İçerik: Ay: Ocak | Bölge: Marmara | Ürün: Ürün A | Satış Adedi: 152 | Ciro (₺): 96520 | Hedef Ciro (₺): 105220.588970937 | Kâr (₺): 28608.41105272736...

[Chunk 2]
Dosya: Aylik_Satis_Ozet_Ornek_Sablon.xlsx
Sayfa: 1
Yetki: ['admin', 'user1']
İçerik: Ay: Ocak | Bölge: Marmara | Ürün: Ürün B | Satış Adedi: 238 | Ciro (₺): 52360 | Hedef Ciro (₺): 48757.8272027132 | Kâr (₺): 9487.574616960714...

[Chunk 3]
Dosya: Aylik_Satis_Ozet_Ornek_Sablon.xlsx
Sayfa: 1
Yetki: ['admin', 'user1']
İçerik: Ay: Ocak | Bölge: Marmara | Ürün: Ürün C | Satış Adedi: 380 | Ciro (₺): 250040 | Hedef Ciro (₺): 268351.736697913 | Kâr (₺): 67566.55950725438...

[Chunk 4]
Dosya: Aylik_Satis_Ozet_Ornek_Sablon.xlsx
Sayfa: 1
Yetki: ['admin', 'user1']
İçerik: Ay: Ocak | Bölge: Marmara | Ürün: Ürün D | Satış Adedi: 201 | Ciro (₺): 66330 | Hedef Ciro (₺): 59970.07390132811 | Kâr (₺): 22816.32409878102...

[Chunk 5]
Dosya: